In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Concatenate, Activation

In [ ]:
input_shape = (299, 299, 3)
classes_num = 1000

In [ ]:
def conv2d_bn(x, filters, kernel_size,
              padding='same', stride=1, activation='relu'):
  x = Conv2D(filters, kernel_size, stride, padding)(x)
  x = BatchNormalization()(x)
  return Activation(activation)(x)

In [ ]:
def inception_module_a(x, filters, name=None):
  filter_a, filter_b, filter_c, filter_d = filters

  path_a = conv2d_bn(x, filter_a[0], 1)
  path_a = conv2d_bn(path_a, filter_a[1], 3)
  path_a = conv2d_bn(path_a, filter_a[2], 3)

  path_b = conv2d_bn(x, filter_b[0], 1)
  path_b = conv2d_bn(path_b, filter_b[1], 3)

  path_c = AveragePooling2D(3, 1, 'same')(x)
  path_c = conv2d_bn(path_c, filter_c, 1)

  path_d = conv2d_bn(x, filter_d, 1)

  return Concatenate(axis=-1, name=name)([path_a, path_b, path_c, path_d])

In [ ]:
def inception_module_b(x, filters, n=7, name=None):
  filter_a, filter_b, filter_c, filter_d = filters

  path_a = conv2d_bn(x, filter_a[0], 1)
  path_a = conv2d_bn(path_a, filter_a[1], (1,n))
  path_a = conv2d_bn(path_a, filter_a[2], (n,1))
  path_a = conv2d_bn(path_a, filter_a[3], (1,n))
  path_a = conv2d_bn(path_a, filter_a[4], (n,1))

  path_b = conv2d_bn(x, filter_b[0], 1)
  path_b = conv2d_bn(path_b, filter_b[1], (1,n))
  path_b = conv2d_bn(path_b, filter_b[2], (n,1))

  path_c = MaxPooling2D(3, 1, 'same')(x)
  path_c = conv2d_bn(path_c, filter_c, 1)

  path_d = conv2d_bn(x, filter_d, 1)

  return Concatenate(axis=-1, name=name)([path_a, path_b, path_c, path_d])

In [ ]:
def inception_module_c(x, filters, name=None):
  filter_a, filter_b, filter_c, filter_d = filters

  path_a = conv2d_bn(x, filter_a[0], 1)
  path_a = conv2d_bn(path_a, filter_a[1], 3)
  path_a_1 = conv2d_bn(path_a, filter_a[2][0], (1, 3))
  path_a_2 = conv2d_bn(path_a, filter_a[2][1], (3, 1))

  path_b = conv2d_bn(x, filter_b[0], 1)
  path_b_1 = conv2d_bn(path_b, filter_b[1][0], (1, 3))
  path_b_2 = conv2d_bn(path_b, filter_b[1][1], (3, 1))

  path_c = MaxPooling2D(3, 1, 'same')(x)
  path_c = conv2d_bn(path_c, filter_c, 1)

  path_d = conv2d_bn(x, filter_d, 1)
  
  concat = Concatenate(axis=-1, name=name)([
              path_a_1, path_a_2,
              path_b_1, path_b_2,
              path_c,
              path_d
  ])

  return concat

In [ ]:
def grid_size_reduction_a(x, filters, name=None):
  filter_a, filter_b = filters

  path_a = conv2d_bn(x, filter_a[0], 1)
  path_a = conv2d_bn(path_a, filter_a[1], 3)
  path_a = conv2d_bn(path_a, filter_a[2], 3, 'valid', 2)
  
  path_b = conv2d_bn(x, filter_b, 3, 'valid', 2)

  path_c = MaxPooling2D(3, 2)(x)

  return Concatenate(axis=-1, name=name)([path_a, path_b, path_c])

In [ ]:
def grid_size_reduction_b(x, filters, name=None):
    filter_a, filter_b = filters
    
    path_a = conv2d_bn(x, filter_a[0], 1)
    path_a = conv2d_bn(path_a, filter_a[1], 3)
    path_a = conv2d_bn(path_a, filter_a[2], 3)
    path_a = conv2d_bn(path_a, filter_a[3], 3,'valid', 2)
    
    path_b = conv2d_bn(x, filter_b[0], 1)
    path_b = conv2d_bn(path_b, filter_b[1], 3, 'valid', 2)
    
    path_c = MaxPooling2D(3, 2 )(x)
    
    return Concatenate(name=name)([path_a, path_b, path_c])

In [ ]:
def auxiliary(x, name=None):
    x = AveragePooling2D(5, 3)(x) # (17, 17, 768) -> (5, 5, 768)
    x = conv2d_bn(x, 128, 1) # (5, 5, 768) -> (5, 5, 128)
    x = conv2d_bn(x, 1024, 1, padding='valid') # (5, 5, 128) -> (1, 1, 1024)
    x = Flatten()(x) # (1, 1, 1024)
    return Dense(classes_num, 'softmax', name='auxiliary')(x)

In [ ]:
input = keras.Input(input_shape)

In [ ]:
x = conv2d_bn(input, 32, 3, 'valid', 2)
x = conv2d_bn(x, 32, 3, 'valid')
x = conv2d_bn(x, 64, 3)
x = MaxPooling2D(3, 2)(x)
x = conv2d_bn(x, 80, 3, 'valid')
x = conv2d_bn(x, 192, 3, 'valid', 2)
x = conv2d_bn(x, 288, 3)
x = inception_module_a(x, [[64, 96, 96], [48, 64], 64 , 64], 'inception_a_1')
x = inception_module_a(x, [[64, 96, 96], [48, 64], 64 , 64], 'inception_a_2')
x = inception_module_a(x, [[64, 96, 96], [48, 64], 64 , 64], 'inception_a_3')
x = grid_size_reduction_a(x, [[64, 96, 96], 384], 'reduction_a')
x = inception_module_b(x, [[128, 128, 128, 128, 192],
                           [128, 128, 192],
                           192, 192], name='inception_b_1')
x = inception_module_b(x, [[160, 160, 160, 160, 192],
                           [160, 160, 192],
                           192, 192], name='inception_b_2')
x = inception_module_b(x, [[160, 160, 160, 160, 192],
                           [160, 160, 192],
                           192, 192], name='inception_b_3')
x = inception_module_b(x, [[192, 192, 192, 192, 192],
                           [192, 192, 192],
                           192, 192], name='inception_b_4')
x = inception_module_b(x, [[192, 192, 192, 192, 192],
                           [192, 192, 192],
                           192, 192], name='inception_b_5')
aux = auxiliary(x, 'auxiliary')
x = grid_size_reduction_b(x, [[192, 192, 192,192],
                              [192, 320]], 'reduction_b')
x = inception_module_c(x, [[448, 384, [384, 384]],
                           [384, [384, 384]],
                           192, 320], name='inception_c_1')
x = inception_module_c(x, [[448, 384, [384, 384]],
                           [384, [384, 384]],
                           192, 320], name='inception_c_2')
x = GlobalAveragePooling2D()(x)
output = Dense(classes_num, 'softmax', name='output')(x)

In [ ]:
model = keras.Model(
    inputs=input,
    outputs=[aux, output]
)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_256 (Conv2D)             (None, 149, 149, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_255 (BatchN (None, 149, 149, 32) 128         conv2d_256[0][0]                 
__________________________________________________________________________________________________
activation_255 (Activation)     (None, 149, 149, 32) 0           batch_normalization_255[0][0]    
______________________________________________________________________________________________

In [ ]:
keras.utils.plot_model(model, show_shapes=True)